# DigiKrawl: DigiKala Comment Crawler

## Prerequisites

The following code snippet import required third-party libraries:
+ `requests` for sending HTTP requests to DigiKala server
+ `bs4` for parsing and crawling HTTP responses
+ `pandas` for data manipulation
+ `datetime` for using time series data
+ `unicode` for decode Farsi characters
+ `jdatetime` for work with Jalali dates

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from unidecode import unidecode
import jdatetime 

DigiKala comments different features. One important one is the date of comment, its format is `'yyyy month_name dd'` so it is necessary to convert them into a proper shape. Following function will do this for you.

In [2]:
def change_date(date):
    """
    date: Jalali date in format '%YYYY %MonthName %dd' string
    return the corresponding Gregorian date
    """
    # Specify number of each month
    months = {"فروردین" : 1,
              "اردیبهشت" : 2,
              "خرداد" : 3,
              "تیر" : 4,
              "مرداد" : 5,
              "شهریور" : 6,
              "مهر" : 7,
              "آبان" : 8,
              "آذر" : 9,
              "دی" : 10,
              "بهمن" : 11,
              "اسفند" : 12,}
    # Extract day
    day = int(unidecode(u'{}'.format(date.split()[0])))
    
    # Extract month
    month = months[date.split()[1]]
    
    # Extract year
    year = int(unidecode(u'{}'.format(date.split()[2])))
    
    # Transform calendar format of the date
    date = jdatetime.date(year,month,day).togregorian()
        
    return date

## Features
We write this function to extract features of data. In this case, features are exactly from book category. But if you wish to crawl another category, you need to change these features.

In [30]:
def crawl_product(url, features):
    """
    url: The product of DigiKala that must be crawled
    features: The features of the output dataframe
    returns pandas.DataFrame containing all comments
    """
    # Define deafult features and combine them with desired ones
    features = ['نام', 'قیمت', 'امتیاز', 'تعداد نظرات'] + features
    
    # Send request to get data as a soup
    response = requests.get(url, timeout=5)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Define a dictionary for product for ease of insertion
    features_dict = {feature:None for feature in features}
    
    # Try to find desired feature
    for feature in features:
        key_node = soup.find('span', text=feature)
        if(key_node):
            features_dict[feature] = key_node.parent.next_sibling.text.strip()
            
    # Find default features
    features_dict['نام'] = soup.find('h1', {'class':'c-product__title'}).text.strip()
    price = soup.find('div', {'class':'c-product__seller-price-pure js-price-value'})
    if (price):
        features_dict['قیمت'] = price.text.strip()  
    engagement = soup.find('div', {'class':'c-product__engagement-rating'})
    if (engagement):
        engagement = engagement.text
        features_dict['امتیاز'] = engagement.split('(')[0].strip()
        features_dict['تعداد نظرات'] = engagement.split('(')[1].strip()[:-1]

    return pd.DataFrame([features_dict])
    

In [4]:
url = 'https://www.digikala.com/product/dkp-2623047/کتاب-چهار-اثر-از-فلورانس-اسکاول-شین-اثر-فلورانس-اسکاول-شین-انتشارات-نگین-ایران'
features = ['وزن', 'نویسنده', 'قطع', 'نوع جلد', 'نوع کاغذ', 'ناشر', 'مترجم', 'تعداد صفحه', 'تعداد صفحه', 'موضوع', 'رده‌بندی کتاب', 'شابک']
s = crawl_product(url, features)
s

,نام,قیمت,امتیاز,تعداد نظرات,وزن,نویسنده,قطع,نوع جلد,نوع کاغذ,ناشر,مترجم,تعداد صفحه,موضوع,رده‌بندی کتاب,شابک
0,کتاب چهار اثر از فلورانس اسکاول شین اثر فلوران...,None,۳.۶,۴۸۱۰,400 گرم,فلورانس اسکاول شین,رقعی,شومیز,تحریر,نگین ایران,فرشاد قدیری,400,روانشناسی,None,9786006230887


## Comments
Then we need to find the data in product's page. For this purpose, we will create next function.

In [6]:
def crawl_digikala_comments(url, pages = 1, mode = 1):
    """
    url: The product of DigiKala that must be crawled
    page: Number of pages of comments, by default it will crawl only first page
    mode: mode = 1 is for newest comment in webpage, mode = 2 is for most liked comments
    returns pandas.DataFrame containing all comments
    """
    # Configure mode of comments
    if (mode == 1):
        mode_str = 'newest_comment'
    elif (mode == 2):
        mode_str = 'most_liked'
    
    # Send request to the specified url
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find product_id and product_title from url and http response respectively
    product_id = url.split('/')[4][4:]
    product_title = soup.find('h1', {'class':'c-product__title'}).text.strip()
    
    # Initialize DataFrame for comments consists of 13 main features
    data = pd.DataFrame(columns = {'Product_id', 'Product_title', 'Title', 'Date', 'Username', 'Badge', 'Status', 'Content',
                                           'Advantages', 'Disadvantages', 'Color', 'Seller', 'Likes'})
    
    # Iterate each page of comments
    for page in range(pages):
        # Modify url for comments and send request
        comment_url = 'https://www.digikala.com/ajax/product/comments/list/{0}/?mode={1}&page={2}'.format(product_id, mode_str, page+1)
        response = requests.get(comment_url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find each comment for further process
        comment_section = soup.find('div', {'class' : 'c-comments__list'})
        # Check for available section
        if(not comment_section):
            break
        comments = comment_section.findChildren('div', {'class' : 'c-comments__item c-comments__item--pdp'})
        
        # Iterate each comment in the page
        for comment in comments:
            # Find title of the commnet
            title = comment.find('span', {'class' : 'c-comments__title'}).text.strip()
            
            # Find string of date of the comment
            date = comment.find_all('span', {'class' : 'c-comments__detail'})[0].text
            # Transform string date into datetime.date instance
            date = change_date(date)
            
            # Find username of the comment's writer
            username = comment.find_all('span', {'class' : 'c-comments__detail'})[1].text.strip()
            
            # Find user badge if exists
            badge = comment.find('div', {'class' : 'c-comments__buyer-badge'})
            if badge is not None:
                badge = badge.text
                
            # Find status of the comment if exists
            status = comment.find('div', {'class' : 'c-comments__status'})
            if status is not None:
                status = status.text
                
            # Find content of the comment
            content = comment.find('div', {'class' : 'c-comments__content'}).text
            
            # Find mentioned advantages in the comment 
            advantages = comment.find_all('div', {'class' : 'c-comments__modal-evaluation-item--positive'})            
            advantages = [advantage.text for advantage in advantages]
                        
            # Find mentioned disadvantages in the comment
            disadvantages = comment.find_all('div', {'class' : 'c-comments__modal-evaluation-item--negative'})
            disadvantages = [disadvantage.text.strip() for disadvantage in disadvantages]
            
            # Find color of the products if exists
            color = comment.find('div', {'class' : 'c-comments__color'})
            if color is not None:
                color = color.text.strip()
                
            # Find the seller's name if exists
            seller = comment.find('a', {'class' : 'c-comments__seller'})
            if seller is not None:
                seller = seller.text.strip()
            
            # Find the number of likes and convert into English digits 
            likes = unidecode(u'{}'.format(comment.find('div', {'class' : 'c-comments__helpful-yes js-comment-like'}).text))
                        
            # Append new commnet into DataFrame
            data = data.append({'Product_id':product_id, 'Product_title': product_title, 'Title': title,
                                'Date': date, 'Username': username, 'Badge': badge, 'Status': status,
                                'Content': content, 'Advantages': advantages, 'Disadvantages': disadvantages,
                                'Color': color, 'Seller': seller, 'Likes':likes}, ignore_index = True)

            
    return data

Let's look at a little example.

In [7]:
url = "https://www.digikala.com/product/dkp-49877/%DA%A9%D8%AA%D8%A7%D8%A8-%D9%84%D8%B7%D9%81-%D8%AA%D9%85%D8%A8%DA%A9-%D8%A2%D9%85%D9%88%D8%B2%D8%B4-%D8%B1%DB%8C%D8%AA%D9%85-%D8%A8%D9%87-%DA%A9%D9%88%D8%AF%DA%A9%D8%A7%D9%86-%D8%A8%D9%87-%DA%A9%D9%85%DA%A9-%D8%B4%D8%B9%D8%B1-%D8%A7%D8%AB%D8%B1-%D9%84%DB%8C%D9%84%D8%A7-%D8%AD%DA%A9%DB%8C%D9%85-%D8%A7%D9%84%D9%87%DB%8C"
data_digikala = crawl_digikala_comments(url,1,2)
data_digikala

,Disadvantages,Status,Product_title,Date,Color,Seller,Likes,Content,Badge,Product_id,Advantages,Username,Title
0,[],در مورد خرید این محصول مطمئن نیستم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2020-06-27,None,موسیقی بتهوون,\n 1\n ...,قیمت زیادی داره ولی مجبور به خرید بودم,خریدار,49877,[],مهسا خادم,خوب
1,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2019-10-05,None,موسیقی بتهوون,\n 1\n ...,خیلی گرونه البته من چاره ای نداشتم باید برای ک...,خریدار,49877,[],آزاده زمانی,گرون
2,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2021-10-01,None,استریو تومبا,\n 0\n ...,کتاب مناسب برای آموزش,خریدار,49877,[],هانا احمدوزیر,کتاب لطف تمبک
3,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2021-06-21,None,گودی کالا,\n 0\n ...,خوب بود,خریدار,49877,[],کاربر دیجی‌کالا,
4,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2021-04-16,None,آراکس,\n 0\n ...,من توی تخفیف خریدم ممنون از دیجی کالا,خریدار,49877,[],محمد علیانی,
5,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2021-01-19,None,گودی کالا,\n 0\n ...,مشکلی نداشت و خوبه,خریدار,49877,[],بیتا ر,راضیم
6,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2020-12-07,None,گودی کالا,\n 0\n ...,برای شروع تنبک عالیه,خریدار,49877,[],نرگس اشتری,راضی بودم
7,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2020-11-26,None,گودی کالا,\n 0\n ...,برای آموزش تمبک به کودکان مناسب,خریدار,49877,[],ابوالفضل صادقیانپور,برای آموزش تمبک به کودکان مناسب
8,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2020-11-10,None,گودی کالا,\n 0\n ...,لازم داشتیم اینجا راحت در دسترس بود \r\nمنون,خریدار,49877,[],آزاده دادستان,کتاب لطف تنبک
9,[],خرید این محصول را توصیه می‌کنم,کتاب لطف تمبک، آموزش ریتم به کودکان به کمک شعر...,2020-07-22,None,گودی کالا,\n 0\n ...,کیفیت چاپ و نشر خوب و مصور و قابل قبوله و خود ...,خریدار,49877,[],ژیلا فضلی,کیفیت آموزش


## Extract Links for Automation
So for so good! But we don't want to manually call this funciton for each product so it is useful to extract list of all products in the same category and automatically call it for them.

In [8]:
def crawl_digikala_list(base_url, from_page=1, to_page=1, sort=4):
    """
    url: The list of DigiKala that must be crawled
    from_page: Number of page to start crawling
    from_page: Number of page to finish crawling
    mode: mode = 4 is for most viewed, mode = 7 is for most purchased
    returns pandas.DataFrame containing all comments
    """
    
    # Define default dataframe
    data = pd.DataFrame(columns = {'Product_title', 'url'})
    
    # Iterate over pages
    for page in range(from_page, to_page+1):
        # Request to get list of products in current page
        url = ('{0}?pageno={1}&sortby={2}').format(base_url, page, sort)
        try:
            print('Requesting...', url)
            response = requests.get(url, timeout=5)
        except:
            return data
        
        # Find Products in json tree
        response_body = response.json()['data']['products'].encode('utf-8').decode('utf-8')

        soup = BeautifulSoup(response_body)
        
        # Find products in soup
        products = soup.find_all('div', {'class': 'c-product-box__content'})

        # Iterate over products in current page and store url in the dataframe
        for product in products:
            url_holder = product.find('a', href=True)
            link = 'https://www.digikala.com' + url_holder['href']
            title = url_holder.text
            
            data = data.append({'Product_title': title, 'url':link}, ignore_index=True)
    
    return data
            


## Putting All Together
Putting these function together, we can easily extract the comments on products on the same category. For ease of running, it will just crawl and save two first prodcuts, but feel free to change it to your desired number.

Note that it would be nice if you add `time.sleep(1000)` between the requests to avoid to apply any pressure on DigiKala servers.

In [9]:
# Following codes were meant to crawl and store/load data in a file

# products_list.to_csv('List.csv', encoding='utf-8-sig', index=False)
products_list = pd.read_csv('List.csv', encoding='utf-8-sig')
products_list

,url,Product_title
0,https://www.digikala.com/product/dkp-5079161/ک...,کتاب اثر مرکب اثر دارن هاردی انتشارات نگین ایران
1,https://www.digikala.com/product/dkp-2623047/ک...,کتاب چهار اثر از فلورانس اسکاول شین اثر فلوران...
2,https://www.digikala.com/product/dkp-5547149/ک...,کتاب معجزه شکرگزاری اثر راندا برن انتشارات نگی...
3,https://www.digikala.com/product/dkp-3963589/ک...,کتاب باشگاه پنج صبحی ها اثر رابین شارما نشر آز...
4,https://www.digikala.com/product/dkp-2892053/ک...,کتاب معجزه شکرگزاری اثر راندا برن انتشارات نیک...
...,...,...
9967,https://www.digikala.com/product/dkp-243132/کت...,کتاب چگونه فیلمنامه بنویسیم 2 اثر سید فیلد
9968,https://www.digikala.com/product/dkp-2904813/ک...,کتاب فوتبالیست ها آماده نبرد اثر علیرضا شریفی ...
9969,https://www.digikala.com/product/dkp-151316/کت...,کتاب پوشک دیگه بسه اثر فانی جولی
9970,https://www.digikala.com/product/dkp-4536111/ک...,کتاب نهضت شعوبیه اثر حسینعلی ممتحن نشر علمی فر...


In following two cells, we will use function all together to crawl entire data. Note that it can consume a lot of time!

In [ ]:
# Store Comments
result = pd.DataFrame()

for url in products_list['url']:
    print('Requesting...', url)
    data = crawl_digikala_comments(url, pages=4, mode=2)
    result = result.append(data)

# Uncomment for saving file
# result.to_csv('DigiKalaComments.csv', index = False, encoding='utf-8-sig')
# result.head()

In [ ]:
# Store Features
result = pd.DataFrame()
features = ['وزن', 'نویسنده', 'قطع', 'نوع جلد', 'نوع کاغذ', 'ناشر', 'مترجم', 'تعداد صفحه', 'تعداد صفحه', 'موضوع', 'رده‌بندی کتاب', 'شابک']

for i, url in enumerate(products_list['url'][9586:]):
    print('Request {}...'.format(i), url)
    data = crawl_product(url, features)
    result = result.append(data)

# Uncomment for saving file
# result.to_csv('DigiKalaProducts.csv', index = False, encoding='utf-8-sig')
# result.head()